In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

#funcion para sacar la informaicon de los departamanetos
def extract_data_from_page(soup):
    search_layout = soup.find('ol', class_='ui-search-layout ui-search-layout--grid')
    if not search_layout:
        print("No se encontró la lista de propiedades en esta página.")
        return []

    page_data = []
    for item in search_layout.find_all('li', class_='ui-search-layout__item'):
        try:
            content_div = item.find('div', class_='ui-search-result__content no-borders')

            # Nombre de la propiedad
            title_label_grid = content_div.find('div', class_='ui-search-item__title-label-grid')
            property_name = title_label_grid.get_text(strip=True) if title_label_grid else "No disponible"

            # Precio
            price_group = content_div.find('div', class_='ui-search-item__group ui-search-item__group--price')
            price_element = price_group.find('div', class_='ui-search-price ui-search-price--size-medium ui-search-item__group__element') if price_group else None
            price_line = price_element.find('div', class_='ui-search-price__second-line ui-search-price__second-line--decimal') if price_element else None
            price_span = price_line.find('span', class_='andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript') if price_line else None
            price = price_span.get_text(strip=True) if price_span else "No disponible"

            # Atributos
            attributes_list = content_div.find('ul', class_='ui-search-card-attributes ui-search-item__group__element ui-search-item__attributes-grid')
            dormitorios, banos, metros_cuadrados = "No disponible", "No disponible", "No disponible"
            if attributes_list:
                attributes = attributes_list.find_all('li', class_='ui-search-card-attributes__attribute')
                if len(attributes) > 0:
                    dormitorios = attributes[0].get_text(strip=True)
                if len(attributes) > 1:
                    banos = attributes[1].get_text(strip=True)
                if len(attributes) > 2:
                    metros_cuadrados = attributes[2].get_text(strip=True)

            # Dirección
            direction_group = content_div.find('div', class_='ui-search-item__group__element ui-search-item__location-container')
            direction_depa = direction_group.find('div', class_='ui-search-item__group ui-search-item__group--location') if direction_group else None
            direction_name = direction_depa.get_text(strip=True) if direction_depa else "No disponible"

            # Guardar los datos
            page_data.append({
                "Nombre Propiedad": property_name,
                "Precio": price,
                "Dormitorios": dormitorios,
                "Baños": banos,
                "Metros Cuadrados": metros_cuadrados,
                "Dirección": direction_name
            })
        except AttributeError:
            continue

    return page_data


In [4]:
#procedemos a ver la indentacion de todos los departamentos

lista_de_comunas = ['','nunoa-metropolitana','la-florida-metropolitana','centro-historico-de-santiago-santiago-santiago-metropolitana']

for comuna in lista_de_comunas:
    data_ = []
    base_url = f'https://www.portalinmobiliario.com/arriendo/departamento/{comuna}' if comuna else 'https://www.portalinmobiliario.com/arriendo/departamento/'

    incrementos_paginas = 48
    start_index = 1
    number_page = 42
    current_page = 1


    while current_page <= number_page:
        url = f"{base_url}_Desde_{start_index}_NoIndex_Tue"
        print(f"Procesando Pagina{current_page}")

        response = requests.get(url)
        if response.status_code != 200:
            print(f"No se pudo conectar con la pagina{current_page} Codigo {response.status_code}")
            break

        soup = BeautifulSoup(response.text,'html.parser')
        page_data = extract_data_from_page(soup)

        if not page_data:
            print(f"no se encontraron datos en {current_page}. Finalizando scrapping")
            break

        data_.extend(page_data)

        time.sleep(1)

        start_index += incrementos_paginas
        current_page +=1
    
    comuna_nombre = comuna.replace("-",'_') if comuna else "todas las comunas"
    df = pd.DataFrame(data_)
    df.to_csv(f'departamentos_{comuna_nombre}.csv', index=False)
    print((f"Datos exportados para la comuna: {comuna or 'Todas las comunas'}"))

    

Procesando Pagina1
Procesando Pagina2
Procesando Pagina3
Procesando Pagina4
Procesando Pagina5
Procesando Pagina6
Procesando Pagina7
Procesando Pagina8
Procesando Pagina9
Procesando Pagina10
Procesando Pagina11
Procesando Pagina12
Procesando Pagina13
Procesando Pagina14
Procesando Pagina15
Procesando Pagina16
Procesando Pagina17
Procesando Pagina18
Procesando Pagina19
Procesando Pagina20
Procesando Pagina21
Procesando Pagina22
Procesando Pagina23
Procesando Pagina24
Procesando Pagina25
Procesando Pagina26
Procesando Pagina27
Procesando Pagina28
Procesando Pagina29
Procesando Pagina30
Procesando Pagina31
Procesando Pagina32
Procesando Pagina33
Procesando Pagina34
Procesando Pagina35
Procesando Pagina36
Procesando Pagina37
Procesando Pagina38
Procesando Pagina39
Procesando Pagina40
Procesando Pagina41
Procesando Pagina42
Datos exportados para la comuna: Todas las comunas
Procesando Pagina1
Procesando Pagina2
Procesando Pagina3
Procesando Pagina4
Procesando Pagina5
Procesando Pagina6
Proc